In [1]:
import tensorflow as tf
import os
from keras import Model
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Flatten, Dropout, UpSampling2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D,Dropout, DepthwiseConv2D
from keras.utils.image_utils import img_to_array, array_to_img, img_to_array, load_img
from keras.optimizers import Adam, SGD
from keras import models
from keras.callbacks import CSVLogger
from keras.models import load_model
import matplotlib.pyplot as plt
import keras.backend as K
import cv2
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

In [8]:
ADAM_LEARNING_RATE = 0.0001

class_labels=['Angry','Contemp', 'Disgust','Fear','Happy','Neutral','Sad','Surprise']

# Bring the custom data

def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc'),
      f1_score,
]

adam = Adam(learning_rate= ADAM_LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [9]:
# Import model
# model = load_model('VGG_Transfer/VGG16_tf.h5', compile=False)
model = load_model('InceptionResNetV2_tf_224_grayscale.h5', compile=False)


model.compile(optimizer=adam, loss='categorical_crossentropy',metrics=METRICS)

face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [10]:
folder = "../TEST_IMAGES/"

for filename in os.listdir(folder):
    # Load the image
    frame = cv2.imread(os.path.join(folder, filename))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces= face_classifier.detectMultiScale(gray, 1.3, 3)
    for (x,y,w,h) in faces:

        sub_face_img=gray[y:y+h, x:x+w]
        resized=cv2.resize(sub_face_img,(224,224))
        normalize=resized/255.0
        reshaped=np.reshape(normalize, (1, 224, 224, 1))
        result=model.predict(reshaped)
        label=np.argmax(result, axis=1)[0]
        print(label)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 1)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(50,50,255),2)
        cv2.rectangle(frame,(x,y-40),(x+w,y),(50,50,255),-1)
        cv2.putText(frame, class_labels[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    cv2.imshow("Frame",frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

1/1 [==============================] - 2s 2s/step
4
1/1 [==============================] - 0s 84ms/step
1
1/1 [==============================] - 0s 41ms/step
7
1/1 [==============================] - 0s 42ms/step
0
1/1 [==============================] - 0s 49ms/step
3
1/1 [==============================] - 0s 42ms/step
4
1/1 [==============================] - 0s 41ms/step
4
1/1 [==============================] - 0s 48ms/step
3
1/1 [==============================] - 0s 44ms/step
2
1/1 [==============================] - 0s 44ms/step
0
1/1 [==============================] - 0s 42ms/step
2
1/1 [==============================] - 0s 44ms/step
2
1/1 [==============================] - 0s 45ms/step
3
1/1 [==============================] - 0s 43ms/step
0
1/1 [==============================] - 0s 40ms/step
3
1/1 [==============================] - 0s 40ms/step
2
1/1 [==============================] - 0s 64ms/step
2
1/1 [==============================] - 0s 42ms/step
0
1/1 [=========================

In [ ]:
gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


faces= face_classifier.detectMultiScale(gray, 1.3, 3)


for (x,y,w,h) in faces:

    sub_face_img=gray[y:y+h, x:x+w]
    resized=cv2.resize(sub_face_img,(224,224))
    normalize=resized/255.0
    reshaped=np.reshape(normalize, (1, 224, 224, 1))
    result=model.predict(reshaped)
    label=np.argmax(result, axis=1)[0]
    print(label)
    cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 1)
    cv2.rectangle(frame,(x,y),(x+w,y+h),(50,50,255),2)
    cv2.rectangle(frame,(x,y-40),(x+w,y),(50,50,255),-1)
    cv2.putText(frame, class_labels[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
cv2.imshow("Frame",frame)
cv2.waitKey(0)
cv2.destroyAllWindows()